In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import tqdm
import pandas as pd
import numpy as np

In [ ]:
import os

res0_names = os.listdir('/content/drive/MyDrive/antifraud/dataset/img2text0.csv')
res1_names = os.listdir('/content/drive/MyDrive/antifraud/dataset/img2text1.csv')
res0_test_names = os.listdir('/content/drive/MyDrive/antifraud/test/img2text0.csv')
res1_test_names = os.listdir('/content/drive/MyDrive/antifraud/test/img2text1.csv')

In [ ]:
df0 = df0.fillna('')
df1 = df1.fillna('')

In [ ]:
item_meta0 = df0

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
model.cuda()

In [ ]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

In [ ]:
batch_size = 64
n = len(item_meta0)
content_item_embeddings0 = []

In [ ]:
with torch.no_grad():
    for i in tqdm.tqdm(range(0, n, batch_size)):
        sentences = item_meta0['Text'].iloc[i:min(i + batch_size, n)].values.tolist()
        sentence_embeddings = embed_bert_cls(sentences, model, tokenizer)
        content_item_embeddings0.append(sentence_embeddings)

In [ ]:
item_meta1 = df1

In [ ]:
batch_size = 64
n = len(item_meta1)
content_item_embeddings1 = []

In [ ]:
with torch.no_grad():
    for i in tqdm.tqdm(range(0, n, batch_size)):
        sentences = item_meta1['Text'].iloc[i:min(i + batch_size, n)].values.tolist()
        sentence_embeddings = embed_bert_cls(sentences, model, tokenizer)
        content_item_embeddings1.append(sentence_embeddings)

In [ ]:
embeddings0 = []
for batch in content_item_embeddings0:
  for elem in batch:
    embeddings0.append(elem)

In [ ]:
embeddings1 = []
for batch in content_item_embeddings1:
  for elem in batch:
    embeddings1.append(elem)

In [ ]:
df0['embeddings'] = embeddings0

In [ ]:
df1['embeddings'] = embeddings1

In [ ]:
for i in range(312):
  df0[str(i)] = df0.embeddings.apply(lambda x: x[i]);

In [ ]:
for i in range(312):
  df1[str(i)] = df1.embeddings.apply(lambda x: x[i]);

In [ ]:
df0_emb = df0.drop(columns=['embeddings'])
df1_emb = df1.drop(columns=['embeddings'])

In [ ]:
df0_emb.to_csv('/content/drive/MyDrive/antifraud/dataset/df0_with_embeddings.csv', index=False)
df1_emb.to_csv('/content/drive/MyDrive/antifraud/dataset/df1_with_embeddings.csv', index=False)

In [ ]:
df0_test = df0_test.fillna('')
df1_test = df1_test.fillna('')

In [ ]:
item_test0 = df0_test

In [ ]:
batch_size = 64
n = len(item_test0)
content_test_embeddings0 = []

In [ ]:
with torch.no_grad():
    for i in tqdm.tqdm(range(0, n, batch_size)):
        sentences = item_test0['Text'].iloc[i:min(i + batch_size, n)].values.tolist()
        sentence_embeddings = embed_bert_cls(sentences, model, tokenizer)
        content_test_embeddings0.append(sentence_embeddings)

In [ ]:
item_test1 = df1_test

In [ ]:
batch_size = 64
n = len(item_test1)
content_test_embeddings1 = []

In [ ]:
with torch.no_grad():
    for i in tqdm.tqdm(range(0, n, batch_size)):
        sentences = item_test1['Text'].iloc[i:min(i + batch_size, n)].values.tolist()
        sentence_embeddings = embed_bert_cls(sentences, model, tokenizer)
        content_test_embeddings1.append(sentence_embeddings)

In [ ]:
test_embeddings0 = []
for batch in content_test_embeddings0:
  for elem in batch:
    test_embeddings0.append(elem)

In [ ]:
test_embeddings1 = []
for batch in content_test_embeddings1:
  for elem in batch:
    test_embeddings1.append(elem)

In [ ]:
df0_test['embeddings'] = test_embeddings0
df1_test['embeddings'] = test_embeddings1

In [ ]:
for i in range(312):
  df0_test[str(i)] = df0_test.embeddings.apply(lambda x: x[i])
  df1_test[str(i)] = df1_test.embeddings.apply(lambda x: x[i])

In [ ]:
df0_emb_test = df0_test.drop(columns=['embeddings'])
df1_emb_test = df1_test.drop(columns=['embeddings'])

In [ ]:
df0_emb_test.to_csv('/content/drive/MyDrive/antifraud/test/df0_test_with_embeddings.csv', index=False)
df1_emb_test.to_csv('/content/drive/MyDrive/antifraud/test/df1_test_with_embeddings.csv', index=False)